In [11]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from json import loads
from requests import get
import re
from langdetect import detect

In [12]:
# connect to the database

dotenv_path = os.path.join("..", "scraping", ".env")
load_dotenv(dotenv_path)
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"

engine = create_engine(DATABASE_URL)

query = "SELECT * FROM job_postings WHERE description IS NOT NULL;"
original_df = pd.read_sql(query, engine)
df = original_df.copy()

query2 =  """
    SELECT job_sources.*
    FROM job_sources 
    INNER JOIN job_postings AS j 
    ON job_sources.job_id = j.job_id
    WHERE j.description IS NOT NULL;
"""
original_df2 = pd.read_sql(query2, engine)
df2 = original_df2.copy()

In [13]:
df.head()

,id,job_id,title,location,salary,experience_level,job_type,employment_type,description,responsibilities,requirements,skills,benefits,company,description_criteria,description_en,language
0,14048,4115475043,Senior AI Engineer proficient in Swift,"Warsaw, Mazowieckie, Poland (On-site)",None,None,None,None,We are seeking a highly motivated and experien...,None,None,None,None,"AIDA projektai, MB",Seniority level Mid-Senior level Employment ty...,None,en
1,292656,4180645574,Specjalista ds. UX/UI,"Suchy Las, Wielkopolskie, Poland (On-site)",None,None,None,None,Zakres Obowiązków: Tworzenie rozwiązań z zakr...,None,None,None,None,GOMEZ,Seniority level Entry level Employment type Fu...,Responsibilities: creating solutions in the fi...,pl
2,131329,4174430877,Junior SuccessFactors Time Management Consultant,"Katowice, Śląskie, Poland (Hybrid)",None,None,None,None,SD Worx is a leading European provider of Payr...,None,None,None,None,SD Worx,Seniority level Entry level Employment type Co...,None,en
3,14889,4152982535,GCP Data Engineer with Airflow,Poland (Remote),None,None,None,None,"Hello, let’s meet! We are Xebia - a place whe...",None,None,None,None,Xebia Poland,Seniority level Mid-Senior level Employment ty...,None,en
4,18467,4056842997,Senior Cloud Native Developer,Poland (Remote),None,None,None,None,We are looking for a Senior Cloud Native Devel...,None,None,None,None,EPAM Systems,Seniority level Mid-Senior level Employment ty...,None,en


In [14]:
df2.head()

,id,job_id,source,job_url,date_posted,is_active
0,1,4158353862,LinkedIn,https://www.linkedin.com/jobs/view/4158353862,2025-02-21,True
1,2,4138407547,LinkedIn,https://www.linkedin.com/jobs/view/4138407547,2025-02-21,True
2,3,4144364769,LinkedIn,https://www.linkedin.com/jobs/view/4144364769,2025-02-21,True
3,4,4083706042,LinkedIn,https://www.linkedin.com/jobs/view/4083706042,2025-02-21,True
4,5,4154765012,LinkedIn,https://www.linkedin.com/jobs/view/4154765012,2025-02-21,True


In [15]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

In [16]:


def split_text(text, max_length=500):
    """Splits text into chunks, ensuring each is under the max_length."""
    sentences = re.split(r'\.\s*', text)
    chunks = []
    temp_chunk = ""

    for sentence in sentences:
        if temp_chunk and len(temp_chunk) + len(sentence) + 1 > max_length:
            chunks.append(temp_chunk.strip())
            temp_chunk = sentence
        else:
            temp_chunk = f"{temp_chunk} {sentence}".strip() if temp_chunk else sentence
    if temp_chunk:
        chunks.append(temp_chunk.strip())
    
    return chunks


def translate_text(text):
    """Translates text using the free Google API."""
    url = f"https://translate.googleapis.com/translate_a/single?client=gtx&dt=t&sl=pl&tl=en&q={text}"
    try:
        response = get(url)
        translated_json = loads(response.text)
        return translated_json[0][0][0] if translated_json else text
    except Exception as e:
        print(f"Error translating: {e}")
        return text

def translate_full_text(text):
    """Splits, translates, and reconstructs the translated text."""
    chunks = split_text(text)
    translated_chunks = [translate_text(chunk) for chunk in chunks]
    return " ".join(translated_chunks)


In [17]:
from tqdm import tqdm
from sqlalchemy.exc import OperationalError
import time
import sys
import math
tqdm.pandas()

with engine.connect() as connection:
    connection.execute(text("ALTER TABLE job_postings ADD COLUMN IF NOT EXISTS description_en TEXT;"))
    connection.execute(text("ALTER TABLE job_postings ADD COLUMN IF NOT EXISTS language VARCHAR(10);"))

    connection.commit()

df = pd.read_sql("SELECT * FROM job_postings WHERE description IS NOT NULL;", engine)
df["language"] = df["language"].fillna('')


missing_language_rows = df[df["language"] == ""]
if not missing_language_rows.empty:
    print(f"Detecting language for {len(missing_language_rows)} descriptions...")
    df.loc[df["language"] == "", "language"] = df.loc[df["language"] == "", "description"].apply(detect_language)

    with engine.begin() as conn:
        for _, row in df[df["language"] != ""].iterrows():
            conn.execute(
                text("UPDATE job_postings SET language = :language WHERE job_id = :job_id"),
                {"language": row["language"], "job_id": row["job_id"]}
            )

df["description_en"] = df.get("description_en", pd.Series([None]*len(df)))

df_to_translate = df[(df["language"] == "pl") & (df["description_en"].isnull())].copy()
total = len(df_to_translate)
print(f"Total Polish descriptions to translate: {total}")

skip = False
if total < 5:
    print("No Polish descriptions to translate.")
    skip = True

if not skip:
    chunk_size = math.ceil(total / 10)
    
    for i in range(0, total, chunk_size):
        chunk = df_to_translate.iloc[i:i+chunk_size].copy()
        print(f"\n🔸 Translating chunk {i//chunk_size + 1}/10")
    
        chunk["translated_description"] = chunk["description"].progress_apply(translate_full_text)
    
        # Save to DB
        for _, row in chunk.iterrows():
            try:
                with engine.begin() as conn:
                    conn.execute(
                        text("UPDATE job_postings SET description_en = :desc_en WHERE job_id = :job_id"),
                        {"desc_en": row["translated_description"], "job_id": row["job_id"]}
                    )
            except OperationalError as e:
                print(f"OperationalError for job_id={row['job_id']}: {e}")
                time.sleep(3)
    
        print(f"Chunk {i//chunk_size + 1} saved to database.")


Total Polish descriptions to translate: 0
No Polish descriptions to translate.


In [18]:
df["final_description"] = df.apply(lambda row: row["description_en"] if pd.notnull(row["description_en"]) else row["description"], axis=1)

In [19]:
df.head()

,id,job_id,title,location,salary,experience_level,job_type,employment_type,description,responsibilities,requirements,skills,benefits,company,description_criteria,description_en,language,final_description
0,14048,4115475043,Senior AI Engineer proficient in Swift,"Warsaw, Mazowieckie, Poland (On-site)",None,None,None,None,We are seeking a highly motivated and experien...,None,None,None,None,"AIDA projektai, MB",Seniority level Mid-Senior level Employment ty...,None,en,We are seeking a highly motivated and experien...
1,292656,4180645574,Specjalista ds. UX/UI,"Suchy Las, Wielkopolskie, Poland (On-site)",None,None,None,None,Zakres Obowiązków: Tworzenie rozwiązań z zakr...,None,None,None,None,GOMEZ,Seniority level Entry level Employment type Fu...,Responsibilities: creating solutions in the fi...,pl,Responsibilities: creating solutions in the fi...
2,131329,4174430877,Junior SuccessFactors Time Management Consultant,"Katowice, Śląskie, Poland (Hybrid)",None,None,None,None,SD Worx is a leading European provider of Payr...,None,None,None,None,SD Worx,Seniority level Entry level Employment type Co...,None,en,SD Worx is a leading European provider of Payr...
3,14889,4152982535,GCP Data Engineer with Airflow,Poland (Remote),None,None,None,None,"Hello, let’s meet! We are Xebia - a place whe...",None,None,None,None,Xebia Poland,Seniority level Mid-Senior level Employment ty...,None,en,"Hello, let’s meet! We are Xebia - a place whe..."
4,18467,4056842997,Senior Cloud Native Developer,Poland (Remote),None,None,None,None,We are looking for a Senior Cloud Native Devel...,None,None,None,None,EPAM Systems,Seniority level Mid-Senior level Employment ty...,None,en,We are looking for a Senior Cloud Native Devel...


In [20]:
df.drop(labels=["description", "description_en"], axis=1, inplace=True)
df.columns

Index(['id', 'job_id', 'title', 'location', 'salary', 'experience_level',
       'job_type', 'employment_type', 'responsibilities', 'requirements',
       'skills', 'benefits', 'company', 'description_criteria', 'language',
       'final_description'],
      dtype='object')

In [21]:
df["final_description"].iloc[0]

"We are seeking a highly motivated and experienced AI-Focused Developer with expertise in multiple modal ingestion and inference, as well as experience working on-device mobile applications like OpenAI's Whisper voice-to-text tool. The ideal candidate will have a strong problem-solving mindset and a passion for leveraging AI technologies to build impactful solutions.   Our primary focus with AI is to accurately and helpfully answer questions related to hundreds of city documents, news updates (posted on the client's resource), events, images, etc. Additional details about this focus will be provided during the interview process.   About the project:: It's a centralized communication platform revolutionizing how municipalities engage with residents. The web-based platform and downloadable app streamline fragmented communication channels by consolidating updates, events, and interactions into one seamless experience.   IMPORTANT: the ability to overlap with Mountain Standard Time (North 

In [22]:
df[df["description_criteria"].isnull()].index


Index([807, 5777, 5778, 14211, 21586, 21950, 22530, 26060, 26766, 27233, 27396,
       28021],
      dtype='int64')

In [23]:
df.iloc[4913]


id                                                                 290907
job_id                                                         4153816246
title                   Front-end Engineering Manager/Lead | Poland | ...
location                             Warsaw, Mazowieckie, Poland (Hybrid)
salary                                                               None
experience_level                                                     None
job_type                                                             None
employment_type                                                      None
responsibilities                                                     None
requirements                                                         None
skills                                                               None
benefits                                                             None
company                                                          Deloitte
description_criteria    Seniority leve

In [24]:
df.drop(index=df[df["description_criteria"].isnull()].index, inplace=True)
df.iloc[4913]

id                                                                 287952
job_id                                                         4187445702
title                                            Senior Angular Developer
location                             Warsaw, Mazowieckie, Poland (Hybrid)
salary                                                               None
experience_level                                                     None
job_type                                                             None
employment_type                                                      None
responsibilities                                                     None
requirements                                                         None
skills                                                               None
benefits                                                             None
company                                             Alter Solutions Group
description_criteria    Seniority leve

In [25]:
def extract_job_type(text):
    result = []
    if "remote" in text.lower():
        result.append("Remote")
    elif "hybrid" in text.lower():
        result.append("Hybrid")
    elif "on-site" in text.lower() or "office" in text.lower():
        result.append("On-site")
    return ", ".join(result) if result else None


df["job_type"] = df["location"].apply(extract_job_type)
df["job_type"].value_counts()

job_type
Hybrid     9553
Remote     8981
On-site    8612
Name: count, dtype: int64

In [26]:
print(df["job_type"].isnull().sum())

df["final_description"][df["job_type"].isnull()].apply(extract_job_type).value_counts()


2395


final_description
Remote     765
Hybrid     527
On-site    466
Name: count, dtype: int64

In [27]:
df["job_type"] = df["final_description"][df["job_type"].isnull()].apply(extract_job_type)

In [28]:
def extract_employment_type(text):
    result = []
    if "contract" in text.lower():
        result.append("Contract")
    if "full-time" in text.lower():
        result.append("Full-time")
    if "part-time" in text.lower():
        result.append("Part-time")
    if "b2b" in text.lower():
        result.append("B2B")
    
    return ", ".join(result) if result else None

df["employment_type"] = df["description_criteria"].apply(extract_employment_type)
df["employment_type"].value_counts()


employment_type
Full-time              25685
Contract                2515
Part-time                412
Contract, Full-time        2
Name: count, dtype: int64

In [29]:
print(df["employment_type"].isnull().sum())

df["final_description"][df["employment_type"].isnull()].apply(extract_employment_type).value_counts()

927


final_description
Contract                          162
Contract, Full-time                42
Full-time                          33
Contract, B2B                      13
B2B                                 8
Part-time                           8
Contract, Part-time                 6
Full-time, Part-time                6
Contract, Full-time, Part-time      5
Name: count, dtype: int64

In [30]:

df["employment_type"] = df["final_description"][df["employment_type"].isnull()].apply(extract_employment_type)

In [31]:
df["location"].head(50)

0             Warsaw, Mazowieckie, Poland (On-site)
1        Suchy Las, Wielkopolskie, Poland (On-site)
2                Katowice, Śląskie, Poland (Hybrid)
3                                   Poland (Remote)
4                                   Poland (Remote)
5                                   Poland (Remote)
6             Warsaw, Mazowieckie, Poland (On-site)
7                                   Poland (Remote)
8                           European Union (Remote)
9              Warsaw, Mazowieckie, Poland (Remote)
10             Warsaw, Mazowieckie, Poland (Hybrid)
11             Warsaw, Mazowieckie, Poland (Hybrid)
12             Warsaw, Mazowieckie, Poland (Remote)
13            Warsaw, Mazowieckie, Poland (On-site)
14             Cracow, Małopolskie, Poland (Remote)
15            Warsaw, Mazowieckie, Poland (On-site)
16               Gdańsk, Pomorskie, Poland (Hybrid)
17             Cracow, Małopolskie, Poland (Hybrid)
18                                  Poland (Remote)
19          